In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from PIL import Image, ImageOps
from scipy.misc import imresize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


test
train



In [2]:
train_data_dir = '../input/train'
test_data_dir = '../input/test'

In [3]:
z = glob.glob(train_data_dir + '/*/*.png')
ori_label = []
ori_imgs = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    ori_label.append(fn.split('/')[-2])
    new_img = Image.open(fn)
    ori_imgs.append(ImageOps.fit(new_img, (224, 224), Image.ANTIALIAS).convert('RGB'))


In [4]:
z = glob.glob(test_data_dir + '/*.png')
ori_test_names = []
ori_test_imgs = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    ori_test_names.append(fn.split('/')[-1])
    new_img = Image.open(fn)
    ori_test_imgs.append(ImageOps.fit(new_img, (224, 224), Image.ANTIALIAS).convert('RGB'))

In [5]:
test_imgs = np.array([np.array(im) for im in ori_test_imgs])

In [6]:
test_imgs = test_imgs.reshape(test_imgs.shape[0], 224, 224, 3) / 255

In [7]:
ori_imgs[0], ori_label[0]

(<PIL.Image.Image image mode=RGB size=224x224 at 0x7FA632B2A128>,
 'Black-grass')

In [ ]:
imgs = np.array([np.array(im) for im in ori_imgs])
imgs.shape

In [9]:
imgs = imgs.reshape(imgs.shape[0], 224, 224, 3) / 255
imgs.shape

(4750, 224, 224, 3)

In [10]:
lb = LabelBinarizer().fit(ori_label)
label = lb.transform(ori_label) 

In [11]:
trainX, validX, trainY, validY = train_test_split(imgs, label, test_size=0.05, random_state=42)

In [12]:
uni_labs = np.unique(ori_label, return_counts=True)

In [ ]:
img_width, img_height = 224, 224

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(trainX)

In [ ]:
!cp ../input/resnet50/resnet* ~/.keras/models/

In [ ]:
base_model = resnet50.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling='avg', classes=12)

In [ ]:
add_model = Sequential()
add_model.add(Flatten(input_shape=(224, 224, 3)))
#inputShape = (224, 224, 3)
#model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
#model.add(Activation("relu"))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#model.add(Conv2D(50, (5, 5), padding="same"))
#model.add(Activation("relu"))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#model.add(Flatten())
add_model.add(Dense(256))
add_model.add(Activation("relu"))

add_model.add(Dense(12))
add_model.add(Activation("softmax"))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])

#m = model.fit_generator(datagen.flow(trainX, trainY, batch_size=32), steps_per_epoch=len(trainX) /32, epochs=32)

In [ ]:
m = model.fit_generator(datagen.flow(trainX, trainY, batch_size=32), steps_per_epoch=len(trainX) /32, epochs=32)

In [ ]:
test = model.fit(imgs, label, validation_split=0.2, epochs=10, batch_size=10)

In [ ]:
prediction = test.predict(test_imgs)


In [ ]:
prediction.shape

In [ ]:
prediction

In [ ]:
predicition2 = model.predict_classes(test_imgs)

In [ ]:
counter = 0
test = uni_labs[0]
result = pd.DataFrame(columns=['file', 'species'])
for i in ori_test_names:
    pred = predicition2[counter]
    result = result.append({'file': i, 'species': test[pred]}, ignore_index=True)
    counter = counter + 1
    
result.to_csv('submission.csv', index=False)

#pred = predicition2[0]
#test = uni_labs[0]
#test[pred]
#lab = lb.inverse_transform([1])